# Tools and Routing

- Functions and services an LLM can utilize to extend its capabilities are named "tools" in LangChain
- LangChain has many tools available.
  - search tools
  - Math tools
  - SQL tools
  -...

In this lab, it focuses on:
1. create yoru won tools;
2. build a toold based on an openapi spec:
  - predating LLMs, the openapi specification is routinely used by service providers to describe their APIs.
3. select from multiple possible toools - called "routing" (ask llm to choose the tools)

In [1]:
!pip install -q openai langchain python-dotenv
!pip install -q wikipedia
!pip install -q requests

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
OPENAI_API_KEY = openai.api_key

In [3]:
from langchain.agents import tool

In [4]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [5]:
search.name

'search'

In [6]:
search.description

'Search for weather online'

In [7]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [8]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")
## description of the input is to tell the LLM how the input should be.
# therefore, it is important to define what the SearchInput would be.

In [9]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [10]:
search.args

{'query': {'description': 'Thing to search for',
  'title': 'Query',
  'type': 'string'}}

In [11]:
search.run("sf")

'42f'

In [12]:
import requests
from pydantic import BaseModel, Field
import datetime


# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")


# self defined function that will be sent to llm to be used at the right time.

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

# getting the forcast for the next day and get the forecast that is closer to the current time.
    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'

In [13]:
get_current_temperature.name

'get_current_temperature'

In [14]:
get_current_temperature.description
# doc string

'Fetch current temperature for given coordinates.'

In [15]:
get_current_temperature.args

{'latitude': {'description': 'Latitude of the location to fetch weather data for',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude of the location to fetch weather data for',
  'title': 'Longitude',
  'type': 'number'}}

In [16]:
from langchain.tools.render import format_tool_to_openai_function

# convert this tool to an openai function

In [17]:
format_tool_to_openai_function(get_current_temperature)
# we get a json, that is possibel for openai to undestand as a function with a chain.

<ipython-input-17-4be3c6878d9e>:1: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  format_tool_to_openai_function(get_current_temperature)


{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [18]:
get_current_temperature({"latitude": 15, "longitude": 14})

<ipython-input-18-df5bf01ecf0c>:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_current_temperature({"latitude": 15, "longitude": 14})


'The current temperature is 28.8°C'

## let's define some other functions....

In [117]:
class CurrencyConversionInput(BaseModel):
    from_currency: str = Field(..., description="the currency to convert from, USD or CNY)")
    to_currency: str = Field(..., description="the currency to convert to, USD or CNY)")
    amount: float = Field(..., description="he amount of money to convert")

@tool(args_schema=CurrencyConversionInput)
def convert_currency(from_currency: str, to_currency: str, amount: float) -> str:
  # must have a docstring if it does not have description....
    """Convert currency from one type to another using an exchange rate API."""

    BASE_URL = f"https://api.exchangerate-api.com/v4/latest/{from_currency}"

    response = requests.get(BASE_URL)

    if response.status_code == 200:
        data = response.json()
        exchange_rate = data['rates'][to_currency]
        converted_amount = amount * exchange_rate
        return f"{amount} {from_currency} is equal to {converted_amount:.2f} {to_currency}."
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")


In [119]:
convert_currency.name

'convert_currency'

In [122]:
convert_currency.description

'Convert currency from one type to another using an exchange rate API.'

In [123]:
convert_currency.args

{'from_currency': {'description': 'the currency to convert from (e.g., USD, CNY)',
  'title': 'From Currency',
  'type': 'string'},
 'to_currency': {'description': 'the currency to convert to (e.g., USD, CNY)',
  'title': 'To Currency',
  'type': 'string'},
 'amount': {'description': 'he amount of money to convert',
  'title': 'Amount',
  'type': 'number'}}

In [124]:
format_tool_to_openai_function(convert_currency)
# we get a json, that is possibel for openai to undestand as a function with a chain.

{'name': 'convert_currency',
 'description': 'Convert currency from one type to another using an exchange rate API.',
 'parameters': {'properties': {'from_currency': {'description': 'the currency to convert from (e.g., USD, CNY)',
    'type': 'string'},
   'to_currency': {'description': 'the currency to convert to (e.g., USD, CNY)',
    'type': 'string'},
   'amount': {'description': 'he amount of money to convert',
    'type': 'number'}},
  'required': ['from_currency', 'to_currency', 'amount'],
  'type': 'object'}}

In [133]:
# must call the tool with structured input, aka "{}"!
input_data = {
    "from_currency": "USD",
    "to_currency": "CNY",
    "amount": 100.0
}
result = convert_currency(input_data)
print(result)
# corret


100.0 USD is equal to 705.00 CNY.


In [134]:
# must call the tool with structured input
input_data = {
    "from_currency": "CNY",
    "to_currency": "USD",
    "amount": 100.0
}
result = convert_currency(input_data)
print(result)
# corret


100.0 CNY is equal to 14.20 USD.


In [19]:
import wikipedia
# self-defined function.. later the langchain will send this to a llm and let the llm
# decide when it is a good time to use this function to answer questions.
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    # get the first three element of that page
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [20]:
search_wikipedia.name

'search_wikipedia'

In [21]:
search_wikipedia.description

'Run Wikipedia search and get page summaries.'

In [22]:
format_tool_to_openai_function(search_wikipedia)
# convert to an open ai fucntion format

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search and get page summaries.',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [23]:
search_wikipedia({"query": "langchain"})

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval augmented generation (RAG) is a type of generative artificial intelligence that has information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information in preference to information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  \nUse cases include providing chatbot access to internal company data, or giving factual information only from an authoritative source.\n\nPage: DataStax\nSummary: D

In [24]:
search_wikipedia({"query": "The Substance (movie)"})

## indeed the first paragraphs of the wiki contents

"Page: The Substance\nSummary: The Substance is a 2024 satirical body horror film co-produced, written and directed by Coralie Fargeat. The film stars Demi Moore, Margaret Qualley and Dennis Quaid. An international co-production between the United Kingdom, the United States and France, its plot follows a fading celebrity (Moore) who decides to use a black market drug, a cell-replicating substance that temporarily creates a younger, better version of herself (Qualley), unknowingly giving herself horrifying side effects.\nThe Substance premiered on 19 May 2024 at the 77th Cannes Film Festival, and was selected to compete for the Palme d'Or in its main competition section, where Fargeat won Best Screenplay. It was released theatrically in the United Kingdom and the United States by Mubi on 20 September 2024, and is scheduled to be released in France by Metropolitan Filmexport on 6 November 2024.\n\nPage: Arkansas (film)\nSummary: Arkansas is a 2020 American neo-noir crime thriller film di

## so far we've done is to create a function in notebook and then create an openai function using the function we just created.
## often times, fucntions we want to interact with apis
## so now, we can use open apis to interact with openai function calls

### this section not working very well.... because some package issues

In [100]:
# !pip uninstall langchain langchain-community langsmith
# !pip install langchain==0.3.0 langchain-community==0.3.0 langsmith==0.1.125
# !pip show langchain langchain-community langsmith
# !pip install langchain==0.0.312
# !pip install pydantic==1.10.8

In [101]:
# !pip install -U langchain-community
# !pip install openapi-schema-pydantic==1.2.4
# !pip install langchain==0.0.312

In [43]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec


In [27]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [28]:
# !pip install --upgrade pydantic==1.2

In [29]:
# openapi-schema-pydantic = {version = "^1.2", optional = true}

In [38]:
# # has to implement these lines of codes so the next line of code does not throw erros:
# # source: https://github.com/langchain-ai/langchain/issues/9520
# @classmethod
# def parse_obj(cls, obj: dict) -> OpenAPISpec:
#     try:
#         cls._alert_unsupported_spec(obj)
#         return super().parse_obj(obj)
#     except ValidationError as e:
#         # We are handling possibly misconfigured specs and
#         # want to do a best-effort job to get a reasonable interface out of it.
#         new_obj = copy.deepcopy(obj)
#         for error in e.errors():
#             keys = error["loc"]
#             item = new_obj
#             for key in keys[:-1]:
#                 item = item[key]
#             item.pop(keys[-1], None)
#         return cls.parse_obj(new_obj)

In [31]:
# !pip install openapi-schema-pydantic==1.2.4
# !pip install pydantic==1.10.8
# !pip install langchain==0.0.312
# the only working way to solve this issue

In [32]:
# llm = openai(
#       model_name="gpt-4",
#       temperature=0,
#       openai_api_key= OPENAI_API_KEY,
#   )

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# analytics_toolkit = NLAToolkit.from_llm_and_spec(
#       llm, OpenAPISpec.from_spec_dict(api_spec)
#   )

# !pip install --upgrade langchain_community

In [33]:
# alternative way to parse openai

# !pip install openapi-spec-validator


In [34]:
# from openapi_spec_validator import validate_spec
# import json

# openapi_spec = json.loads(text)
# validate_spec(openapi_spec)


In [83]:
# !pip install openapi-schema-pydantic==1.2.4


In [68]:
# spec = OpenAPISpec.from_text(text)
# # spec = OpenAPISpec.from_text(text)

# # # AttributeError: 'super' object has no attribute 'parse_obj'

In [ ]:
# pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(openapi_spec)

In [ ]:
# pet_openai_functions

In [84]:
# from langchain.chat_models import ChatOpenAI

In [66]:
# from langchain.chat_models import ChatOpenAI


In [85]:
# !pip uninstall langchain
# !pip install langchain==0.3.0

In [ ]:
# model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [ ]:
# model.invoke("what are three pets names")

In [ ]:
# model.invoke("tell me about pet with id 42")

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [102]:
from langchain.chat_models import ChatOpenAI


In [135]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature, convert_currency
    ]
]
model = ChatOpenAI(temperature=0, openai_api_key = OPENAI_API_KEY).bind(functions=functions)

In [71]:
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 105, 'total_tokens': 130, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-02b09d7b-cb65-4607-a1b6-4ca55d7ded59-0')

In [72]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 101, 'total_tokens': 117, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-2c38c099-d648-4b49-b7c3-189c14327502-0')

In [136]:
model.invoke("how much is 100 USD worth in CNY?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"from_currency":"USD","to_currency":"CNY","amount":100}', 'name': 'convert_currency'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 185, 'total_tokens': 210, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-0abcbb20-2267-4b39-b88d-738cbb80d8ac-0')

In [137]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [74]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 113, 'total_tokens': 138, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-d0b9ccac-9bed-418f-bb3a-fcbf8edc5f63-0')

In [138]:
# convert_currency

chain.invoke({"input": "How much is 100 CNY worth in USD?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"from_currency":"CNY","to_currency":"USD","amount":100}', 'name': 'convert_currency'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 194, 'total_tokens': 219, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-c2a3c363-b562-4aad-8c85-0f65f94b587d-0')

In [75]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [76]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [139]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [78]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [79]:
result.tool

'get_current_temperature'

In [80]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [81]:
get_current_temperature(result.tool_input)

'The current temperature is 14.0°C'

In [82]:
result = chain.invoke({"input": "hi!"})

In [86]:
type(result)

langchain_core.agents.AgentFinish

In [87]:
result.return_values

{'output': 'Well, hello there! How can I assist you today?'}

In [144]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
            "convert_currency": convert_currency

        }
        return tools[result.tool].run(result.tool_input)

In [145]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [146]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [91]:
result

'The current temperature is 14.0°C'

In [103]:
result = chain.invoke({"input": "What is langchain?"})

In [104]:
result2 = chain.invoke({"input": "Tell me something about the movie 'the Substance"})

In [93]:
result

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval augmented generation (RAG) is a type of generative artificial intelligence that has information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information in preference to information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  \nUse cases include providing chatbot access to internal company data, or giving factual information only from an authoritative source.\n\nPage: DataStax\nSummary: D

In [105]:
result2

# will automatically get to the wiki page

"Page: The Substance\nSummary: The Substance is a 2024 satirical body horror film co-produced, written and directed by Coralie Fargeat. The film stars Demi Moore, Margaret Qualley and Dennis Quaid. An international co-production between the United Kingdom, the United States and France, its plot follows a fading celebrity (Moore) who decides to use a black market drug, a cell-replicating substance that temporarily creates a younger, better version of herself (Qualley), unknowingly giving herself horrifying side effects.\nThe Substance premiered on 19 May 2024 at the 77th Cannes Film Festival, and was selected to compete for the Palme d'Or in its main competition section, where Fargeat won Best Screenplay. It was released theatrically in the United Kingdom and the United States by Mubi on 20 September 2024, and is scheduled to be released in France by Metropolitan Filmexport on 6 November 2024.\n\nPage: Arkansas (film)\nSummary: Arkansas is a 2020 American neo-noir crime thriller film di

In [106]:
result3 = chain.invoke({"input": "Tell me something about the new iPhone16"})

In [107]:
result3

# will automatically get to the wiki page

'Page: IPhone 16\nSummary: The iPhone 16  and iPhone 16 Plus are smartphones designed, developed, and marketed by Apple Inc. They are the eighteenth-generation iPhones, succeeding the iPhone 15 and iPhone 15 Plus. The devices were unveiled alongside the higher-priced iPhone 16 Pro and 16 Pro Max during the September 9, 2024 Apple Event at Apple Park in Cupertino, California.\n\n\n\nPage: IPhone 16 Pro\nSummary: The iPhone 16 Pro and iPhone 16 Pro Max are smartphones designed, developed, and marketed by Apple Inc. Alongside the iPhone 16 and iPhone 16 Plus, they form the eighteenth generation of the iPhone, succeeding the iPhone 15 Pro and iPhone 15 Pro Max, and were announced on September 9, 2024, and released on September 20, 2024. The iPhone 16 Pro and iPhone 16 Pro Max include larger 6.3-inch and 6.9-inch displays, a faster processor, upgraded wide and ultra-wide cameras, support for Wi-Fi 7, larger batteries, and the iOS 18 operating system.\n\n\n\nPage: IPhone 13\nSummary: The iPh

In [109]:
result4 = chain.invoke({"input": "Tell me something about Andrew Ng"})

In [110]:
result4
# will automatically get to the wiki page

'Page: Andrew Ng\nSummary: Andrew Yan-Tak Ng (Chinese: 吳恩達; born 1976) is a British-American computer scientist and technology entrepreneur focusing on machine learning and artificial intelligence (AI). Ng was a cofounder and head of Google Brain and was the former Chief Scientist at Baidu, building the company\'s Artificial Intelligence Group into a team of several thousand people.\nNg is an adjunct professor at Stanford University (formerly associate professor and Director of its Stanford AI Lab or SAIL). Ng has also worked in the field of online education, cofounding Coursera and DeepLearning.AI. He has spearheaded many efforts to "democratize deep learning" teaching over 8 million students through his online courses. Ng is renowned globally in computer science, recognized in Time magazine\'s 100 Most Influential People in 2012 and Fast Company\'s Most Creative People in 2014. His influence extends to being named in the Time100 AI Most Influential People in 2023.\nIn 2018, he launch

In [111]:
result5 = chain.invoke({"input": "What is the weather in New York City right now?"})

In [112]:
result5
# correct, will use the weather function automaticlaly.

'The current temperature is 17.2°C'

In [113]:
result6 = chain.invoke({"input": "How to set up an OpenAI account?"})

In [114]:
result6
# does not directly fit in weather question or wikipedia question...
# will still generate somethin the langauge model knows? but not necessary relevant

'Page: Dead Internet theory\nSummary: The dead Internet theory is an online conspiracy theory that asserts that, due to a coordinated and intentional effort, the Internet now consists mainly of bot activity and automatically generated content manipulated by algorithmic curation to intentionally manipulate the population and minimize organic human activity. Proponents of the theory believe these bots were created intentionally to help manipulate algorithms and boost search results in order to manipulate consumers. Some proponents of the theory accuse government agencies of using bots to manipulate public perception. The date given for this "death" is generally around 2016 or 2017.\nThe dead Internet theory has gained traction because many of the observed phenomena are quantifiable, such as increased bot traffic, but the literature does not support the full theory. Caroline Busta, founder of the media platform New Models, was quoted in an article in The Atlantic calling much of the dead 

In [147]:
result7 = chain.invoke({"input": "What is 100 CNY worth in USD?"})

In [148]:
result7

'100.0 CNY is equal to 14.20 USD.'

In [156]:
result8 = chain.invoke({"input": "What is 200 japanese yen worth in USD?"})

In [157]:
result8

'200.0 JPY is equal to 1.39 USD.'